In [54]:
import requests
import json
import pandas as pd
# import matplotlib.pyplot as plt
import pyarrow as pa

In [28]:
# Get data from an example data set
sample_dataset_url = 'https://www.portlandmaps.com/arcgis/rest/services/Public/COP_OpenData_Miscellaneous/MapServer/1354/query?where=1=1&outFields=*&f=json'

# In the current case, we're looking at neighborhood profiles
neighborhood_profiles_request = requests.get(sample_dataset_url)
if (neighborhood_profiles_request.status_code == 200):
    neighborhood_profiles_json = json.loads(neighborhood_profiles_request.content)
    neighborhood_profiles = pd.DataFrame.from_dict(neighborhood_profiles_json['features'])
    print('Data from Sample Data Set returned successfully. Here are the first 5 rows:')
    print(neighborhood_profiles.head())
else:
    print('URL for Sample Data Set is invalid')

Data from Sample Data Set returned successfully. Here are the first 5 rows:
                                          attributes
0  {'OBJECTID': 1, 'Neighborhood': 'Alameda', 'PO...
1  {'OBJECTID': 2, 'Neighborhood': 'Arbor Lodge',...
2  {'OBJECTID': 3, 'Neighborhood': 'Ardenwald-Joh...
3  {'OBJECTID': 4, 'Neighborhood': 'Argay Terrace...
4  {'OBJECTID': 5, 'Neighborhood': 'Arlington Hei...


In [49]:
# Let's apply basic data profiling techniques to the sample data
np = pd.DataFrame(pd.json_normalize(neighborhood_profiles_json['features']))
np.rename(columns=lambda name: name.replace("attributes.", ""), inplace=True)

print('Here is the general shape of the data:')
print(np.describe())

Here is the general shape of the data:
        OBJECTID          POP10           HU10     poplt18_10          POP20  \
count  99.000000      99.000000      99.000000      99.000000      99.000000   
mean   50.000000   12010.484848    5448.939394    2301.525253   13417.707071   
std    28.722813   58285.410691   26490.829538   11171.943241   65139.198846   
min     1.000000       2.000000       1.000000       0.000000      40.000000   
25%    25.500000    2883.500000    1227.000000     361.500000    3156.000000   
50%    50.000000    5346.000000    2319.000000     992.000000    5866.000000   
75%    74.500000    7932.500000    3875.500000    1408.500000    8949.000000   
max    99.000000  583776.000000  265439.000000  111523.000000  652503.000000   

                HU20        HHPOP20        poplt18     NH_NAT20      NH_ASN20  \
count      99.000000      99.000000      99.000000    99.000000     99.000000   
mean     6192.292929   12998.848485    2294.505051    87.343434   1073.151515 

In [98]:
# Isolate features for analysis
neighborhood_populations_notcleaned = np[['Neighborhood', 'POP10', 'POP20']]

# There appears to be a record in the data that represents all of the data summed together; there are also 2 rows with population outliers; remove them
neighborhood_populations = neighborhood_populations_notcleaned[(neighborhood_populations_notcleaned.Neighborhood != 'City of Portland') & (~ neighborhood_populations_notcleaned.Neighborhood.str.startswith('MC Unclaimed'))].copy()

print(neighborhood_populations.describe().T)
print(neighborhood_populations.sort_values('POP20', ascending=False))

       count         mean          std    min      25%     50%      75%  \
POP10   94.0  6435.585106  5203.186857  176.0  3175.75  5395.0  7963.25   
POP20   94.0  7184.914894  5713.536926  183.0  3421.25  5995.0  8997.50   

           max  
POP10  30570.0  
POP20  32852.0  
                  Neighborhood  POP10  POP20
73         Powellhurst-Gilbert  30570  32852
32                   Hazelwood  23462  26700
15  Centennial Community Assn.  23651  25413
45                       Lents  20465  22244
62    Northwest District Assn.  13638  18314
..                         ...    ...    ...
4            Arlington Heights    865   1067
10                   Bridgeton    725    928
88                  Sunderland    718    698
95               Woodland Park    176    202
33               Healy Heights    187    183

[94 rows x 3 columns]


In [103]:
# This cell analyzes the Total Population of the City of Portland in 2010 and 2020
portland2010population = sum(neighborhood_populations['POP10'])
portland2020population = sum(neighborhood_populations['POP20'])
population_growth_number = portland2020population - portland2010population
population_growth_perc = 100 * population_growth_number / portland2010population
print("Portland's Total Population in 2010 was: " + str(portland2010population))
print("Portland's Total Popultaion in 2020 was: " + str(portland2020population))
print("This is a Total Growth of: " + str(population_growth_number) + " or " + str(round(population_growth_perc, 2)) + "%.")

Portland's Total Population in 2010 was: 604945
Portland's Total Popultaion in 2020 was: 675382
This is a Total Growth of: 70437 or 11.64%.


In [104]:
pdx_neighborhood_population_changes = neighborhood_populations.POP20 - neighborhood_populations.POP10
neighborhood_populations['population_difference'] = 100 * pdx_neighborhood_population_changes / neighborhood_populations['POP10']
neighborhood_populations.sort_values('population_difference', ascending=False)

,Neighborhood,POP10,POP20,population_difference
47,Lloyd District Community Assn.,1142,2776,143.082312
68,Pearl District,5997,11019,83.741871
82,South Portland,6631,10782,62.599910
23,Eliot,3556,5537,55.708661
36,Hollywood,1578,2260,43.219265
...,...,...,...,...
69,Piedmont,7047,7033,-0.198666
87,Sumner Association of Neighbors,2137,2118,-0.889097
66,Parkrose Neighborhood,6363,6250,-1.775892
33,Healy Heights,187,183,-2.139037
